In [4]:
from transformers import pipeline

summarizer = pipeline("summarization")


No model was supplied, defaulted to sshleifer/distilbart-cnn-12-6 (https://huggingface.co/sshleifer/distilbart-cnn-12-6)


[{'summary_text': ' This is'}]


In [23]:
print(summarizer(
    "designed and implemented",
    min_length=2, max_length=5))

[{'summary_text': ' Designed and'}]


In [30]:
#https://stackoverflow.com/questions/72486821/summarization-with-huggingface-how-to-generate-one-word-at-a-time
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

# Load model
tokenizer = AutoTokenizer.from_pretrained("sshleifer/distilbart-xsum-1-1")
model = AutoModelForSeq2SeqLM.from_pretrained("sshleifer/distilbart-xsum-1-1")

text = "The branch of Artificial Intelligence, Natural Language Processing (NLP), is all about making machines understand and process human language. Processing human language is not an easy task for machines as machines work with numbers and not text. 💻 NLP is such a vast and widely studied branch of AI that every now and then we hear a new advancement in this domain. Researchers are trying hard to make the machines understand human language and the context behind it."

# Tokenize text
batch = tokenizer(text, return_tensors="pt")

In [31]:
import torch

generated_sequence = torch.tensor([[tokenizer.sep_token_id]])  # initial token

# Generation loop
while True:
    with torch.no_grad():
        output = model(input_ids=batch["input_ids"], decoder_input_ids=generated_sequence)
    next_token_logits = output.logits[:, -1, :]
    next_token_scores = next_token_logits.softmax(dim=-1)

    # Take token with highest probability
    next_token = next_token_scores.argmax().unsqueeze(0).unsqueeze(0)

    # Append token to generated sequence
    generated_sequence = torch.cat((generated_sequence, next_token), dim=1)
    # Stop if EOS token generated
    if (generated_sequence.squeeze()[-1] == tokenizer.eos_token_id):
        break

summary = tokenizer.batch_decode(generated_sequence, skip_special_tokens=True)

In [32]:
summary

[" The world's largest language language in the world, according to a new study."]

In [35]:
print(summarizer(text, min_length=2, max_length=40))

[{'summary_text': ' The branch of Artificial Intelligence, Natural Language Processing (NLP), is all about making machines understand and process human language . Processing human language is not an easy task for machines as machines work'}]
